In [1]:
import os
import cv2
from albumentations import (
    Compose, HorizontalFlip, VerticalFlip,
    RandomBrightnessContrast, GaussianBlur
)

# Augmentation Pipelines
augmentation_type_1 = Compose([
    HorizontalFlip(p=0.5),
    RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
    GaussianBlur(blur_limit=(3, 7), p=0.5)
])

augmentation_type_2 = Compose([
    VerticalFlip(p=0.5),
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, p=0.5)
])

# List of Augmentations
augmentations = [augmentation_type_1, augmentation_type_2]

# Directories
contrast_image_dir = 'dataset/cropped_with_contrast'
no_contrast_image_dir = 'dataset/cropped_no_contrast'
label_dir = 'dataset/labels_cropped'

augmented_contrast_image_dir = 'dataset/augmented_cropped_images_with_contrast'
augmented_no_contrast_image_dir = 'dataset/augmented_cropped_images_no_contrast'
augmented_label_dir = 'dataset/augmented_cropped_labels'

# Create Directories
os.makedirs(augmented_contrast_image_dir, exist_ok=True)
os.makedirs(augmented_no_contrast_image_dir, exist_ok=True)
os.makedirs(augmented_label_dir, exist_ok=True)

def augment_cropped_image(image_path, label_path, output_image_dir, output_label_dir, augmentations):
    """
    Applies multiple augmentation pipelines to each cropped image and saves results.
    """
    print(f"Processing cropped image: {image_path}")
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error reading image: {image_path}")
        return

    base_name = os.path.splitext(os.path.basename(image_path))[0]

    # Read the corresponding label file
    try:
        with open(label_path, 'r') as file:
            label_content = file.readlines()
    except FileNotFoundError:
        print(f"Label file not found for image: {image_path}")
        return

    # Read class ID and bounding box from label file (cropped images only have one bounding box)
    if len(label_content) != 1:
        print(f"Unexpected label format in {label_path}, skipping.")
        return
    class_id, x_center, y_center, width, height = map(float, label_content[0].split())

    # Apply each augmentation pipeline
    for aug_idx, augmentation in enumerate(augmentations):
        augmented = augmentation(image=image)['image']

        # Adjust labels for flips
        adjusted_x_center = x_center
        adjusted_y_center = y_center

        # Horizontal Flip Adjustment
        if isinstance(augmentation, HorizontalFlip):
            adjusted_x_center = 1 - x_center

        # Vertical Flip Adjustment
        if isinstance(augmentation, VerticalFlip):
            adjusted_y_center = 1 - y_center

        # Save augmented image
        aug_image_name = f"{base_name}_aug_{aug_idx}.png"
        aug_image_path = os.path.join(output_image_dir, aug_image_name)
        cv2.imwrite(aug_image_path, augmented)
        print(f"Saved augmented image: {aug_image_path}")

        # Save corresponding annotation file
        aug_label_name = f"{base_name}_aug_{aug_idx}.txt"
        aug_label_path = os.path.join(output_label_dir, aug_label_name)
        with open(aug_label_path, 'w') as label_file:
            label_file.write(f"{int(class_id)} {adjusted_x_center:.6f} {adjusted_y_center:.6f} {width:.6f} {height:.6f}\n")
        print(f"Saved augmented label: {aug_label_path}")

def contains_target_class(label_path):
    """
    Check if the label file contains 'Living' (class_id=1) or 'Bubble' (class_id=2) classes.
    """
    target_classes = {'1', '2'}  # Assuming 1 is 'Living' and 2 is 'Bubble'
    with open(label_path, 'r') as file:
        for line in file:
            class_id = line.split()[0]
            if class_id in target_classes:
                return True
    return False

# Process Cropped Images
for folder, output_image_dir in [
    (contrast_image_dir, augmented_contrast_image_dir),
    (no_contrast_image_dir, augmented_no_contrast_image_dir)
]:
    print(f"\nProcessing folder: {folder}")
    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            image_path = os.path.join(folder, filename)
            label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + '.txt')

            # Apply augmentation only if the label contains 'Living' or 'Bubble' classes
            if contains_target_class(label_path):
                augment_cropped_image(image_path, label_path, output_image_dir, augmented_label_dir, augmentations)


/Users/valentinaolaritei/Desktop/3_SEMESTER/.venv/lib/python3.12/site-packages/albumentations/check_version.py:51: UserWarning: Error fetching version info <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
  data = fetch_version_info()



Processing folder: dataset/cropped_with_contrast
Processing cropped image: dataset/cropped_with_contrast/010-0896_0_shape_70.png
Saved augmented image: dataset/augmented_cropped_images_with_contrast/010-0896_0_shape_70_aug_0.png
Saved augmented label: dataset/augmented_cropped_labels/010-0896_0_shape_70_aug_0.txt
Saved augmented image: dataset/augmented_cropped_images_with_contrast/010-0896_0_shape_70_aug_1.png
Saved augmented label: dataset/augmented_cropped_labels/010-0896_0_shape_70_aug_1.txt
Processing cropped image: dataset/cropped_with_contrast/010-1008_Non-Living_shape_73.png
Saved augmented image: dataset/augmented_cropped_images_with_contrast/010-1008_Non-Living_shape_73_aug_0.png
Saved augmented label: dataset/augmented_cropped_labels/010-1008_Non-Living_shape_73_aug_0.txt
Saved augmented image: dataset/augmented_cropped_images_with_contrast/010-1008_Non-Living_shape_73_aug_1.png
Saved augmented label: dataset/augmented_cropped_labels/010-1008_Non-Living_shape_73_aug_1.txt
P